# AG2: Tools (Tools with Secrets)

- Overview<br>
  - https://docs.ag2.ai/docs/user-guide/basic-concepts/tools/index
- Tools with Secrets<br>
  - https://docs.ag2.ai/docs/user-guide/basic-concepts/tools/tools-with-secrets
- Tools with Dependency Injection<br>
  - https://docs.ag2.ai/docs/use-cases/notebooks/notebooks/tools_dependency_injection

## Overview

Agents in AG2 leverage tools to extend their functionality, allowing them to interact with external systems, fetch real-time data, and execute complex tasks beyond the scope of a language model’s internal knowledge. This enables a structured approach where agents decide which tool to use and then execute it accordingly.
- How Tool Usage Works in AG2
  - **Selection**: An agent (driven by its LLM) decides which tool is appropriate based on the given task.
  - **Execution**: A separate executor agent invokes the tool and returns the results.
- Secure Tool Usage with Secrets
  - **Enhanced Security**: Keeps secrets out of LLM interactions and telemetry.
  - **Simplified Development**: Securely passes credentials without hardcoding them.
  - **Flexible Integration**: Allows agents to access necessary credentials while maintaining security.
- Interoperability with External Frameworks
  - **LangChain**: Provides numerous pre-built tools for API calls, web scraping, and more.
  - **CrewAI**: Offers specialized tools for web scraping, search, and automation.
  - **PydanticAI**: Supports structured data handling and dependency injection for context-driven tool execution.

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## Tools with Secrets

In [2]:
# Imports and an LLM configuration for all agents
import os
from typing import Annotated, Literal

from pydantic import BaseModel

# from autogen import GroupChat, GroupChatManager, LLMConfig
from autogen import GroupChat, GroupChatManager
from autogen.agentchat import ConversableAgent, UserProxyAgent
from autogen.tools.dependency_injection import BaseContext, Depends

# config_list = [{"api_type": "openai", "model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]
config_list = {"api_type": "openai", "model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}

### Mock third party systems

In [3]:
# Mock functions accessing 3rd party systems

def weather_api_call(username: str, password: str, location: str) -> str:
    print(f"Accessing third party Weather System using username {username}")
    return "It's sunny and 40 degrees Celsius in Sydney, Australia."


def my_ticketing_system_availability(username: str, password: str, concert: str) -> bool:
    print(f"Accessing third party Ticketing System using username {username}")
    return False

### Our credentials structure

In [4]:
# Credentials class based on BaseContext
class ThirdPartyCredentials(BaseContext, BaseModel):
    username: str
    password: str

### Agents for each system

In [5]:
# Our two 3rd party system accessing agents
weather_agent = ConversableAgent(
    name="weather_agent",
    system_message="You are a Weather Agent, you can only get the weather.",
    description="Weather Agent solely used for getting weather.",
    # llm_config=LLMConfig(config_list=config_list),
    llm_config=config_list,
)

ticket_agent = ConversableAgent(
    name="ticket_agent",
    system_message="You are a Ticketing Agent, you can only get ticket availability.",
    description="Ticketing Agent solely used for getting ticket availability.",
    # llm_config=LLMConfig(config_list=config_list),
    llm_config=config_list,
)

# Our user proxy agent, used to execute the tools
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    llm_config=False,
    code_execution_config={
        "work_dir": "coding_work",
        "use_docker": False,  # Set to True if you want to use Docker
    }
)

### Creating credentials and tools with dependency injection

In [6]:
# Weather API
weather_account = ThirdPartyCredentials(username="ag2weather", password="wbkvEehV1A")


@user_proxy.register_for_execution()
@weather_agent.register_for_llm(description="Get the weather for a location")
def get_weather(
    location: str,
    credentials: Annotated[ThirdPartyCredentials, Depends(weather_account)],
) -> str:
    # Access the Weather API using the credentials
    return weather_api_call(username=credentials.username, password=credentials.password, location=location)


# Ticketing System API
ticket_system_account = ThirdPartyCredentials(username="ag2tickets", password="EZRIVeVWvA")


@user_proxy.register_for_execution()
@ticket_agent.register_for_llm(description="Get the availability of tickets for a concert")
def tickets_available(
    concert_name: str,
    credentials: Annotated[ThirdPartyCredentials, Depends(ticket_system_account)],
) -> bool:
    return my_ticketing_system_availability(
        username=credentials.username, password=credentials.password, concert=concert_name
    )

### Create Group Chat and run

In [7]:
# Create a GroupChat
groupchat = GroupChat(agents=[user_proxy, weather_agent, ticket_agent], messages=[], max_round=5)
# manager = GroupChatManager(groupchat=groupchat, llm_config=LLMConfig(config_list=config_list))
manager = GroupChatManager(groupchat=groupchat, llm_config=config_list)

# Here's our task for our LLM to help with
message = (
    "Start by getting the weather for Sydney, Australia, and follow that up by checking "
    "if there are tickets for the 'AG2 Live' concert."
)

In [8]:
user_proxy.initiate_chat(manager, message=message, max_turns=1)

user_proxy (to chat_manager):

Start by getting the weather for Sydney, Australia, and follow that up by checking if there are tickets for the 'AG2 Live' concert.

--------------------------------------------------------------------------------

Next speaker: weather_agent


>>>>>>>> USING AUTO REPLY...
weather_agent (to chat_manager):

***** Suggested tool call (call_YZ0hogkX2dHHHh74stbxQbwg): get_weather *****
Arguments: 
{"location":"Sydney, Australia"}
****************************************************************************

--------------------------------------------------------------------------------

Next speaker: user_proxy


>>>>>>>> EXECUTING FUNCTION get_weather...
Call ID: call_YZ0hogkX2dHHHh74stbxQbwg
Input arguments: {'location': 'Sydney, Australia'}
Accessing third party Weather System using username ag2weather
user_proxy (to chat_manager):

***** Response from calling tool (call_YZ0hogkX2dHHHh74stbxQbwg) *****
It's sunny and 40 degrees Celsius in Sydney, Australia

ChatResult(chat_id=None, chat_history=[{'content': "Start by getting the weather for Sydney, Australia, and follow that up by checking if there are tickets for the 'AG2 Live' concert.", 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'None', 'tool_calls': [{'id': 'call_YZ0hogkX2dHHHh74stbxQbwg', 'function': {'arguments': '{"location":"Sydney, Australia"}', 'name': 'get_weather'}, 'type': 'function'}], 'name': 'weather_agent', 'role': 'assistant'}, {'content': "It's sunny and 40 degrees Celsius in Sydney, Australia.", 'tool_responses': [{'tool_call_id': 'call_YZ0hogkX2dHHHh74stbxQbwg', 'role': 'tool', 'content': "It's sunny and 40 degrees Celsius in Sydney, Australia."}], 'role': 'tool', 'name': 'user_proxy'}, {'content': 'None', 'tool_calls': [{'id': 'call_NWjyJ6v3QiYfCWN0gSqZXWT6', 'function': {'arguments': '{"concert_name":"AG2 Live"}', 'name': 'tickets_available'}, 'type': 'function'}], 'name': 'ticket_agent', 'role': 'assistant'}, {'content': 'False', 'tool_responses': [{'t

## More Tool with Dependency Injection examples

Tools with Dependency Injection
- https://docs.ag2.ai/docs/use-cases/notebooks/notebooks/tools_dependency_injection